## Rabi probe
### §PULSE_LENGTH§ns, linearity amp limit at §LINEARITY_AMP_LIMIT§V

In [ ]:
#f.close()
f  = h5py.File(§HDF5_PATH§, 'r', swmr=True)
pa = f['parameters']
da = f['data']
#f.close()

amp   = da['amp'][()]
data1 = da['I1'][()] + 1j * da['Q1'][()]
data2 = da['I2'][()] + 1j * da['Q2'][()] 
data  = IQ_rot((data1 - data2))

#fig, ax = plt.subplots()

#ax.plot(center, freq)
#ax.legend(('f'Freq: linearity_amp_limit = {amp_lim:f}'))

result = {'linearity_amp_limit': 0.249}
globals()['result'] = result
result